In [1]:
!rm -rf ./images
!mkdir images

In [12]:
import os

fonts_folder = "./fonts"
def get_ttf_files(folder):
    ttf_files = []

    for dirpath, dirnames, filenames in os.walk(folder):
        for filename in [f for f in filenames if f.endswith(".ttf")]:
            ttf_files.append(os.path.join(dirpath, filename))

    return ttf_files

ttf_files = get_ttf_files(fonts_folder)
print(ttf_files)

['./fonts/RobotoMono-SemiBoldItalic.ttf', './fonts/Comfortaa-Light.ttf', './fonts/RobotoMono-ThinItalic.ttf', './fonts/RobotoMono-Thin.ttf', './fonts/RobotoMono-SemiBold.ttf', './fonts/Comfortaa-SemiBold.ttf', './fonts/Caveat-Regular.ttf', './fonts/RobotoMono-Bold.ttf', './fonts/Alice-Regular.ttf', './fonts/RobotoMono-ExtraLight.ttf', './fonts/RobotoMono-MediumItalic.ttf', './fonts/Caveat-Medium.ttf', './fonts/RobotoMono-BoldItalic.ttf', './fonts/RobotoMono-Light.ttf', './fonts/RubikDirt-Regular.ttf', './fonts/RobotoMono-Italic.ttf', './fonts/RobotoMono-ExtraLightItalic.ttf', './fonts/Comfortaa-Medium.ttf', './fonts/RobotoMono-Regular.ttf', './fonts/RobotoMono-Medium.ttf', './fonts/RobotoMono-LightItalic.ttf', './fonts/Comfortaa-Regular.ttf', './fonts/Caveat-Bold.ttf', './fonts/Pacifico-Regular.ttf', './fonts/Lobster-Regular.ttf', './fonts/Pangolin-Regular.ttf', './fonts/Comfortaa-Bold.ttf', './fonts/Caveat-SemiBold.ttf']


In [13]:
!mkdir -p ../../PaddleOCR_Mukhtar/train_data/rec/synthetic/img

In [14]:
from trdg.generators import GeneratorFromDict

generator = GeneratorFromDict(
    blur=1,
    length=1,
    random_blur=True,
    random_skew = True,
    size=50,
    skewing_angle=20,
    distorsion_type=3,
    # text_color="#000000",
    # fit=True,
    path="./kk_dict.txt",
    text_colors=["#282828", "#000000", "#333333", "#666666", "#999999"],
    fonts=ttf_files,
    stroke_widths=[0],
    space_widths=[1.0],
    character_spacings=[0],
    stroke_fills=["#282828", "#000000", "#333333", "#666666", "#999999"],
)


In [16]:
from tqdm import tqdm
train_folder = "../../PaddleOCR_Mukhtar/train_data/rec/synthetic/"
relative_train_folder = "./train_data/rec/synthetic/"

# Define train and test split
train_test_split = 1000

image_count = 10000

with open(f"{train_folder}synthetic_test.txt", "w") as test_file, open(f"{train_folder}synthetic_train.txt", "w") as train_file:
    for idx in tqdm(range(image_count)):
        (img, lbl) = generator.next()
        file_name = f"word_{str(idx).zfill(3)}.jpg"
        file_path = f"{train_folder}img/{file_name}"
        img.save(file_path)

        if idx < train_test_split:
            # Write to test file
            test_file.write(f"{file_name}\t{lbl}\n")
        else:
            # Write to train file
            train_file.write(f"{file_name}\t{lbl}\n")


100%|██████████| 10000/10000 [01:50<00:00, 90.42it/s]
